In [ ]:
import pandas as pd
import sqlite3

# Establecer la conexión con la base de datos
conn = sqlite3.connect('argenconsql.db')

# Consulta SQL para seleccionar todos los registros de la tabla argencon
#query = "SELECT * FROM argencon" para seleccionar la tabla entera
# Para seleccionar solo las 2 primeras columnas y le cambio el nombre
query = "SELECT field1 AS Dominios, field2 AS Clases FROM argencon" 

# Cargar los datos en un DataFrame
df = pd.read_sql_query(query, conn)

# Cerrar la conexión con la base de datos
conn.close()

# Ver los primeros registros del DataFrame
print(df.head())

In [ ]:
def asignar_dga(row):
    if 'dga' in row['Clases'].lower():
        return row['Dominios']
    else:
        return None
    
def asignar_normal(row):
    if 'normal' in row['Clases'].lower():
        return row['Dominios']
    else:
        return None    

# Inicializar la columna "DGA" con valores vacíos
df['DGA'] = ''
df['Normal'] = ''

# Guardar los valores retornados por la función en la columna "DGA"
d = 1
n = 1
for i, row in df.iterrows():
    valuedga = asignar_dga(row)
    valuenormal = asignar_normal(row)
    if valuedga:
        df.at[d, 'DGA'] = valuedga
        d = d + 1  
    if valuenormal:
        df.at[n, 'Normal'] = valuenormal
        n = n + 1      
          
# Mostrar el DataFrame resultante
print(df)

In [ ]:
df['Labels'] = df['Clases'].apply(lambda x: 1 if 'dga' in x.lower() else 0)
def obtener_longitud(dominio):
    return len(dominio)

# Aplicar la función a cada valor de la columna "Dominios" y guardar los resultados en la columna "Longitud"
df['Longitud'] = df['Dominios'].apply(obtener_longitud)
df['Longitud'] = df['Longitud']/100
print(df)

In [ ]:
import string

alfabeto = string.ascii_letters + "0123456789-._Ññ"
print(alfabeto)


In [ ]:
# Crear 45 nuevas columnas llenas de 67
for i in range(45):
    columna = f'Tok{i}'  # Nombre de la columna
    df[columna] = 0.67  # Asignar el valor 67 a la columna

# Mostrar el DataFrame resultante
print(df)

In [ ]:
for i, dominio in enumerate(df['Dominios']):
    for j, caracter in enumerate(dominio):
        if j >= 45:
            break  # Salir del bucle si ya se alcanzó la longitud máxima de 45

        columna = f'Tok{j}'  # Nombre de la columna correspondiente al carácter
        valor = alfabeto.index(caracter) if caracter in alfabeto else -1  # Obtener el valor según la posición en el alfabeto o -1 si no está en el alfabeto
        df.at[i, columna] = valor / 100  # Asignar el valor a la celda correspondiente en el DataFrame

# Mostrar el DataFrame resultante
print(df)

In [ ]:
from sklearn.model_selection import train_test_split

# Seleccionar las columnas necesarias para el entrenamiento y prueba
columns =  ['Longitud'] + [f'Tok{i}' for i in range(0, 45)] + ['Labels']
data = df[columns]

# Dividir los datos en conjunto de entrenamiento y prueba
train_data, test_data, train_labels, test_labels = train_test_split(
    data.drop('Labels', axis=1),
    data['Labels'],
    test_size=0.3,
    random_state=42
)

# Verificar los tamaños de los conjuntos
print("Tamaño del conjunto de entrenamiento:", len(train_data))
print("Tamaño del conjunto de prueba:", len(test_data))

# Mostrar el conjunto de entrenamiento
print("Conjunto de entrenamiento:")
print(train_data)

# Mostrar el conjunto de prueba
print("Conjunto de prueba:")
print(test_data)

# Mostrar las etiquetas de entrenamiento
print("Etiquetas de entrenamiento:")
print(train_labels)

# Mostrar las etiquetas de prueba
print("Etiquetas de prueba:")
print(test_labels)



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

timesteps = 46  # Número de pasos de tiempo
features = 1  # Número de características

# Definir la arquitectura del modelo
model = Sequential()
model.add(LSTM(units=64, input_shape=(timesteps, features)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_data, train_labels, epochs=20, batch_size=32, validation_data=(train_data, train_labels))

# Realizar predicciones
predictions = model.predict(test_data)


In [ ]:
import numpy as np
predictions = model.predict(test_data)

# Convertir las predicciones a valores binarios (0 o 1)
binary_predictions = np.round(predictions).flatten()

# Calcular la precisión
accuracy = np.mean(binary_predictions == test_labels)
print("Precisión: ", accuracy)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Redondear las predicciones a 0 o 1
rounded_predictions = np.round(predictions).flatten()

# Calcular las métricas de evaluación
accuracy = accuracy_score(test_labels, rounded_predictions)
precision = precision_score(test_labels, rounded_predictions)
recall = recall_score(test_labels, rounded_predictions)
f1 = f1_score(test_labels, rounded_predictions)

# Calcular la matriz de confusión
confusion_mat = confusion_matrix(test_labels, rounded_predictions)

# Imprimir las métricas y la matriz de confusión
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:")
print(confusion_mat)

In [ ]:
# Supongamos que tu modelo entrenado está en la variable 'model'
model.save('modelo_entrenado_detector.h5')